In [12]:
import pandas as pd
import numpy as np
%matplotlib inline
import plotly 
from wordcloud import WordCloud,STOPWORDS
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import math

#import twitter data
filename = '../Data/Tweets.csv'
dataset = pd.read_csv(filename)
print(dataset.shape)#rows x columns

(14640, 15)


In [13]:
# cantidad de tweet por clase
dataset['airline_sentiment'].value_counts()

negative    9178
neutral     3099
positive    2363
Name: airline_sentiment, dtype: int64

In [14]:
# convertir a entero los valores de las clases
dataset['airline_sentiment'] = dataset['airline_sentiment'].replace('neutral', 1)
dataset['airline_sentiment'] = dataset['airline_sentiment'].replace('negative', 0)
dataset['airline_sentiment'] = dataset['airline_sentiment'].replace('positive', 2)

In [15]:
# Dividar los datos entre text y clase
X = dataset['text'] # data
y = dataset['airline_sentiment'] # labels

<h3>Procesando Datos<7h3>

In [18]:
from tensorflow.keras.preprocessing.text import Tokenizer  #reemplazar los caracteres especiales
from tensorflow.keras.preprocessing.text import text_to_word_sequence #analiza la secuencia de los caracteres
# Convertisa los datos en token
# create tokenizer
t = Tokenizer()
t.fit_on_texts(X)
# Encuentra la cantidad de palabras únicas en los tweets
vocab_size = len(t.word_index) + 1
# indexar las palabras (relacionar con su par entero)
sequences = t.texts_to_sequences(X)

In [19]:
# Encuentra el tweet con mayor numero de caracteres para ser tomado como referencias 
#al momento de vectorizar a las palabras
def max_tweet():
    for i in range(1, len(sequences)):
        max_length = len(sequences[0])
        if len(sequences[i]) > max_length:
            max_length = len(sequences[i])
    return max_length
tweet_num = max_tweet()
print("Mayor numero de caracteres en un tweet " , tweet_num)

Mayor numero de caracteres en un tweet  30


In [20]:
# las palabras no encontradas en el tweet se llenan con ceros en el array
# https://realpython.com/python-keras-text-classification/
from tensorflow.keras.preprocessing.sequence import pad_sequences
maxlen = tweet_num
padded_X = pad_sequences(sequences, padding='post', maxlen=maxlen)
valor_maximo =np.amax(padded_X) 
padded_X

array([[  81,   62, 6686, ...,    0,    0,    0],
       [  81,  558,  590, ...,    0,    0,    0],
       [  81,    3,  207, ...,    0,    0,    0],
       ...,
       [  13,   75,  661, ...,    0,    0,    0],
       [  13,    6,   22, ...,    0,    0,    0],
       [  13,   41,   22, ...,    2,  179,    8]])

In [21]:
# convertir las etiquestas
labels = to_categorical(np.asarray(y))
labels
# 1 => representa la clase que representa negativo / neutral / positivo

array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.]], dtype=float32)

In [22]:
array = dataset.values
padded_Y = array[:,1] #array de clases
padded_Y_test=padded_Y.astype('int')
#padded_Y
padded_X_test = (padded_X /valor_maximo) -1

In [23]:
# Train test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(padded_X, labels, test_size = 0.2, random_state = 0)
# Size of train and test datasets
print('X_train size:', X_train.shape)
print('y_train size:', y_train.shape)
print('X_test size:', X_test.shape)
print('y_test size:', y_test.shape)

X_train size: (11712, 30)
y_train size: (11712, 3)
X_test size: (2928, 30)
y_test size: (2928, 3)


## Pre-trained word embedding

In [25]:
# GloVe se define como un “algoritmo de aprendizaje no supervisado 
#para obtener representaciones vectoriales de palabras”. 
#Descargamos datos del sitio web vinculado y usamos específicamente las incrustaciones
#100-dimensionales de 400k palabras de Wikipedia en inglés en 2014. 
#Esto se representa en un archivo txt que debemos analizar para crear un
#índice que mapee las palabras a su representación vectorial.

# 100 dimensional version (embedding dimension)

#ayuda a en la decodificacion
#https://stackoverflow.com/questions/9233027/unicodedecodeerror-charmap-codec-cant-decode-byte-x-in-position-y-character
embeddings_index = dict()
f = open('../Data/glove/glove.6B.100d.txt', encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Vectores de palabras cargados: %s' % len(embeddings_index))


Vectores de palabras cargados: 400000


In [26]:
# Se Obtiene todas las palabras únicas en nuestro conjunto de entrenamiento: índice de tokenizadores
# Se encuentra el vector de peso correspondiente en la incrustación de GloVe

# Se define el tamaño de la matriz de incrustación: número de palabras únicas x incrustación dim (100)
embedding_matrix = np.zeros((vocab_size, 100))

# llenado de la matrix
for word, i in t.word_index.items():  # dictionary
    embedding_vector = embeddings_index.get(word) # obtiene un vector incrustado de palabra de GloVe
    if embedding_vector is not None:
        # se agrega a la matrix
        embedding_matrix[i] = embedding_vector # cada fila de la matrix

#print(embedding_matrix)
embedding_matrix.shape

(15769, 100)

In [29]:
# Crea una capa de incrustación usando una matriz de incrustación
from tensorflow.keras.layers import Embedding

# la entrada es vocab_size, la salida es 100
# pesos de la matriz de incrustación, establezca entrenable = Falso || para q no se modifiquen los valores de Glove
embedding_layer = Embedding(input_dim=vocab_size, output_dim=100, weights=[embedding_matrix],
                           input_length = tweet_num, trainable=False)

In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import BatchNormalization

### Model 1: Simple LSTM Model with regularization, increase dimensionality

In [36]:
lstm_mod1 = Sequential()
lstm_mod1.add(embedding_layer)
lstm_mod1.add(LSTM(256, 
               dropout = 0.2, 
               recurrent_dropout = 0.5))
lstm_mod1.add(Dense(3, activation='softmax'))
lstm_mod1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
lstm_mod1.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 100)           1576900   
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               365568    
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 771       
Total params: 1,943,239
Trainable params: 366,339
Non-trainable params: 1,576,900
_________________________________________________________________


In [38]:
hist_1 = lstm_mod1.fit(X_train, y_train,
                    validation_split = 0.2,
                    epochs=100, batch_size=256)

Epoch 1/100
37/37 [==============================] - 17s 447ms/step - loss: 0.8293 - acc: 0.6407 - val_loss: 0.7364 - val_acc: 0.6876
Epoch 2/100
37/37 [==============================] - 16s 432ms/step - loss: 0.7038 - acc: 0.7113 - val_loss: 0.6342 - val_acc: 0.7418
Epoch 3/100
37/37 [==============================] - 17s 449ms/step - loss: 0.6420 - acc: 0.7362 - val_loss: 0.6004 - val_acc: 0.7559
Epoch 4/100
37/37 [==============================] - 16s 441ms/step - loss: 0.6178 - acc: 0.7481 - val_loss: 0.5858 - val_acc: 0.7495
Epoch 5/100
37/37 [==============================] - 16s 445ms/step - loss: 0.6037 - acc: 0.7543 - val_loss: 0.5757 - val_acc: 0.7717
Epoch 6/100
37/37 [==============================] - 17s 455ms/step - loss: 0.5857 - acc: 0.7607 - val_loss: 0.5631 - val_acc: 0.7678
Epoch 7/100
37/37 [==============================] - 17s 450ms/step - loss: 0.5718 - acc: 0.7701 - val_loss: 0.5572 - val_acc: 0.7734
Epoch 8/100
37/37 [==============================] - 21s 562ms

Epoch 62/100
37/37 [==============================] - 22s 596ms/step - loss: 0.1410 - acc: 0.9492 - val_loss: 0.8321 - val_acc: 0.7904
Epoch 63/100
37/37 [==============================] - 19s 508ms/step - loss: 0.1321 - acc: 0.9523 - val_loss: 0.9258 - val_acc: 0.7943
Epoch 64/100
37/37 [==============================] - 20s 531ms/step - loss: 0.1374 - acc: 0.9503 - val_loss: 0.8371 - val_acc: 0.7887
Epoch 65/100
37/37 [==============================] - 19s 508ms/step - loss: 0.1299 - acc: 0.9515 - val_loss: 0.9188 - val_acc: 0.7943
Epoch 66/100
37/37 [==============================] - 19s 522ms/step - loss: 0.1386 - acc: 0.9489 - val_loss: 0.8985 - val_acc: 0.7828
Epoch 67/100
37/37 [==============================] - 18s 480ms/step - loss: 0.1282 - acc: 0.9516 - val_loss: 0.8792 - val_acc: 0.7956
Epoch 68/100
37/37 [==============================] - 17s 447ms/step - loss: 0.1249 - acc: 0.9533 - val_loss: 0.8866 - val_acc: 0.7939
Epoch 69/100
37/37 [==============================] - 1

In [42]:
# Entrenamiento y test accuracy
loss, accuracy = lstm_mod1.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = lstm_mod1.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.9530
Testing Accuracy:  0.7722


### Model 2: LSTM with regularization, reduce dimensionality

In [43]:
lstm_mod2 = Sequential()
lstm_mod2.add(embedding_layer)
lstm_mod2.add(LSTM(64, 
               dropout = 0.2, 
               recurrent_dropout = 0.5))
lstm_mod2.add(Dense(3, activation='softmax'))
lstm_mod2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
lstm_mod2.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 100)           1576900   
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                42240     
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 195       
Total params: 1,619,335
Trainable params: 42,435
Non-trainable params: 1,576,900
_________________________________________________________________


In [44]:
hist_2 = lstm_mod2.fit(X_train, y_train,
                    validation_split = 0.2,
                    epochs=100, batch_size=256)

Epoch 1/100
37/37 [==============================] - 5s 146ms/step - loss: 0.8854 - acc: 0.6087 - val_loss: 0.8396 - val_acc: 0.6176
Epoch 2/100
37/37 [==============================] - 4s 112ms/step - loss: 0.8107 - acc: 0.6486 - val_loss: 0.7606 - val_acc: 0.7008
Epoch 3/100
37/37 [==============================] - 4s 117ms/step - loss: 0.7197 - acc: 0.7104 - val_loss: 0.6868 - val_acc: 0.7345
Epoch 4/100
37/37 [==============================] - 4s 105ms/step - loss: 0.6623 - acc: 0.7313 - val_loss: 0.6176 - val_acc: 0.7567
Epoch 5/100
37/37 [==============================] - 4s 114ms/step - loss: 0.6369 - acc: 0.7464 - val_loss: 0.6287 - val_acc: 0.7580
Epoch 6/100
37/37 [==============================] - 4s 110ms/step - loss: 0.6214 - acc: 0.7465 - val_loss: 0.5880 - val_acc: 0.7682
Epoch 7/100
37/37 [==============================] - 4s 103ms/step - loss: 0.5961 - acc: 0.7548 - val_loss: 0.5728 - val_acc: 0.7682
Epoch 8/100
37/37 [==============================] - 4s 106ms/step - 

37/37 [==============================] - 4s 112ms/step - loss: 0.3420 - acc: 0.8630 - val_loss: 0.5757 - val_acc: 0.8058
Epoch 63/100
37/37 [==============================] - 5s 126ms/step - loss: 0.3363 - acc: 0.8691 - val_loss: 0.5710 - val_acc: 0.8011
Epoch 64/100
37/37 [==============================] - 5s 141ms/step - loss: 0.3374 - acc: 0.8705 - val_loss: 0.5463 - val_acc: 0.8075
Epoch 65/100
37/37 [==============================] - 4s 121ms/step - loss: 0.3324 - acc: 0.8685 - val_loss: 0.5750 - val_acc: 0.8058
Epoch 66/100
37/37 [==============================] - 4s 122ms/step - loss: 0.3293 - acc: 0.8709 - val_loss: 0.5586 - val_acc: 0.8015
Epoch 67/100
37/37 [==============================] - 5s 127ms/step - loss: 0.3315 - acc: 0.8672 - val_loss: 0.5673 - val_acc: 0.7981
Epoch 68/100
37/37 [==============================] - 5s 123ms/step - loss: 0.3263 - acc: 0.8692 - val_loss: 0.5844 - val_acc: 0.8054
Epoch 69/100
37/37 [==============================] - 4s 118ms/step - loss:

In [45]:
# Find train and test accuracy
loss, accuracy = lstm_mod2.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = lstm_mod2.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.9143
Testing Accuracy:  0.7968


### Model 3: LSTM Layer Stacking

In [46]:
# LSTM Model
model_3 = Sequential()
model_3.add(embedding_layer)
model_3.add(LSTM(256, 
               dropout = 0.2, 
               recurrent_dropout = 0.5,
                 return_sequences = True))
model_3.add(LSTM(128,
                dropout = 0.2,
                recurrent_dropout = 0.5))
model_3.add(Dense(3, activation='softmax'))
model_3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model_3.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 100)           1576900   
_________________________________________________________________
lstm_5 (LSTM)                (None, 30, 256)           365568    
_________________________________________________________________
lstm_6 (LSTM)                (None, 128)               197120    
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 387       
Total params: 2,139,975
Trainable params: 563,075
Non-trainable params: 1,576,900
_________________________________________________________________


In [47]:
history_3 = model_3.fit(X_train, y_train,
                    validation_split = 0.2,
                    epochs=100, batch_size=256)

Epoch 1/100
37/37 [==============================] - 30s 814ms/step - loss: 0.8122 - acc: 0.6498 - val_loss: 0.7434 - val_acc: 0.7192
Epoch 2/100
37/37 [==============================] - 29s 797ms/step - loss: 0.6764 - acc: 0.7265 - val_loss: 0.6523 - val_acc: 0.7362
Epoch 3/100
37/37 [==============================] - 30s 798ms/step - loss: 0.6228 - acc: 0.7429 - val_loss: 0.6058 - val_acc: 0.7601
Epoch 4/100
37/37 [==============================] - 30s 810ms/step - loss: 0.6093 - acc: 0.7513 - val_loss: 0.5967 - val_acc: 0.7563
Epoch 5/100
37/37 [==============================] - 30s 808ms/step - loss: 0.5929 - acc: 0.7549 - val_loss: 0.5818 - val_acc: 0.7665
Epoch 6/100
37/37 [==============================] - 30s 805ms/step - loss: 0.5781 - acc: 0.7641 - val_loss: 0.5538 - val_acc: 0.7729
Epoch 7/100
37/37 [==============================] - 30s 813ms/step - loss: 0.5644 - acc: 0.7686 - val_loss: 0.5597 - val_acc: 0.7717
Epoch 8/100
37/37 [==============================] - 30s 803ms

Epoch 62/100
37/37 [==============================] - 30s 799ms/step - loss: 0.1052 - acc: 0.9622 - val_loss: 0.9430 - val_acc: 0.7832
Epoch 63/100
37/37 [==============================] - 30s 805ms/step - loss: 0.0942 - acc: 0.9657 - val_loss: 0.9389 - val_acc: 0.7909
Epoch 64/100
37/37 [==============================] - 30s 803ms/step - loss: 0.0972 - acc: 0.9640 - val_loss: 0.9414 - val_acc: 0.7793
Epoch 65/100
37/37 [==============================] - 29s 796ms/step - loss: 0.0973 - acc: 0.9634 - val_loss: 0.9209 - val_acc: 0.7751
Epoch 66/100
37/37 [==============================] - 30s 799ms/step - loss: 0.0969 - acc: 0.9652 - val_loss: 0.9106 - val_acc: 0.7879
Epoch 67/100
37/37 [==============================] - 30s 802ms/step - loss: 0.0957 - acc: 0.9657 - val_loss: 0.9123 - val_acc: 0.7819
Epoch 68/100
37/37 [==============================] - 30s 799ms/step - loss: 0.0928 - acc: 0.9674 - val_loss: 0.9569 - val_acc: 0.7828
Epoch 69/100
37/37 [==============================] - 3

In [48]:
# Find train and test accuracy
loss, accuracy = model_3.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model_3.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.9557
Testing Accuracy:  0.7818


### Model 4: GRU Layer Stacking

In [49]:
# GRU Model 2:
model_4 = Sequential()
model_4.add(embedding_layer)
model_4.add(GRU(256, 
               dropout = 0.2, 
               recurrent_dropout = 0.5,
                 return_sequences = True))
model_4.add(GRU(128,
                dropout = 0.2,
                recurrent_dropout = 0.5))
model_4.add(Dense(3, activation='softmax'))
model_4.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model_4.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 100)           1576900   
_________________________________________________________________
gru (GRU)                    (None, 30, 256)           274944    
_________________________________________________________________
gru_1 (GRU)                  (None, 128)               148224    
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 387       
Total params: 2,000,455
Trainable params: 423,555
Non-trainable params: 1,576,900
_________________________________________________________________


In [50]:
history_4 = model_4.fit(X_train, y_train,
                    validation_split = 0.2,
                    epochs=100, batch_size=256)

Epoch 1/100
37/37 [==============================] - 24s 642ms/step - loss: 0.8663 - acc: 0.6233 - val_loss: 0.7675 - val_acc: 0.6748
Epoch 2/100
37/37 [==============================] - 23s 624ms/step - loss: 0.7340 - acc: 0.6895 - val_loss: 0.7405 - val_acc: 0.6906
Epoch 3/100
37/37 [==============================] - 24s 646ms/step - loss: 0.6866 - acc: 0.7142 - val_loss: 0.6308 - val_acc: 0.7465
Epoch 4/100
37/37 [==============================] - 25s 662ms/step - loss: 0.6264 - acc: 0.7421 - val_loss: 0.6057 - val_acc: 0.7584
Epoch 5/100
37/37 [==============================] - 24s 657ms/step - loss: 0.5893 - acc: 0.7570 - val_loss: 0.5651 - val_acc: 0.7776
Epoch 6/100
37/37 [==============================] - 24s 656ms/step - loss: 0.5721 - acc: 0.7677 - val_loss: 0.5483 - val_acc: 0.7700
Epoch 7/100
37/37 [==============================] - 24s 657ms/step - loss: 0.5625 - acc: 0.7702 - val_loss: 0.5379 - val_acc: 0.7857
Epoch 8/100
37/37 [==============================] - 24s 658ms

Epoch 62/100
37/37 [==============================] - 33s 899ms/step - loss: 0.1257 - acc: 0.9530 - val_loss: 0.8600 - val_acc: 0.7960
Epoch 63/100
37/37 [==============================] - 32s 855ms/step - loss: 0.1210 - acc: 0.9546 - val_loss: 0.7951 - val_acc: 0.7934
Epoch 64/100
37/37 [==============================] - 28s 767ms/step - loss: 0.1232 - acc: 0.9545 - val_loss: 0.8142 - val_acc: 0.7939
Epoch 65/100
37/37 [==============================] - 25s 689ms/step - loss: 0.1169 - acc: 0.9582 - val_loss: 0.8410 - val_acc: 0.7917
Epoch 66/100
37/37 [==============================] - 25s 669ms/step - loss: 0.1118 - acc: 0.9585 - val_loss: 0.8066 - val_acc: 0.8024
Epoch 67/100
37/37 [==============================] - 24s 640ms/step - loss: 0.1063 - acc: 0.9603 - val_loss: 0.8859 - val_acc: 0.7973
Epoch 68/100
37/37 [==============================] - 24s 659ms/step - loss: 0.1080 - acc: 0.9582 - val_loss: 0.8202 - val_acc: 0.8015
Epoch 69/100
37/37 [==============================] - 2

In [51]:
loss, accuracy = model_4.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model_4.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.9544
Testing Accuracy:  0.7865


### Model 5: Reduced GRU with More Regularization

In [52]:
model_5 = Sequential()
model_5.add(embedding_layer)
model_5.add(GRU(64, 
               dropout = 0.3, 
               recurrent_dropout = 0.5,
                 return_sequences = True))
model_5.add(GRU(32,
                dropout = 0.2,
                recurrent_dropout = 0.5))
model_5.add(Dense(3, activation='softmax'))
model_5.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model_5.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 100)           1576900   
_________________________________________________________________
gru_2 (GRU)                  (None, 30, 64)            31872     
_________________________________________________________________
gru_3 (GRU)                  (None, 32)                9408      
_________________________________________________________________
dense_7 (Dense)              (None, 3)                 99        
Total params: 1,618,279
Trainable params: 41,379
Non-trainable params: 1,576,900
_________________________________________________________________


In [53]:
history_5 = model_5.fit(X_train, y_train,
                    validation_split = 0.2,
                    epochs=100, batch_size=256)

Epoch 1/100
37/37 [==============================] - 6s 175ms/step - loss: 0.9157 - acc: 0.6155 - val_loss: 0.8583 - val_acc: 0.6176
Epoch 2/100
37/37 [==============================] - 6s 156ms/step - loss: 0.8469 - acc: 0.6283 - val_loss: 0.8330 - val_acc: 0.6453
Epoch 3/100
37/37 [==============================] - 5s 146ms/step - loss: 0.8045 - acc: 0.6592 - val_loss: 0.8107 - val_acc: 0.6662
Epoch 4/100
37/37 [==============================] - 6s 174ms/step - loss: 0.7450 - acc: 0.6734 - val_loss: 0.7211 - val_acc: 0.6987
Epoch 5/100
37/37 [==============================] - 7s 186ms/step - loss: 0.7025 - acc: 0.7021 - val_loss: 0.6612 - val_acc: 0.7264
Epoch 6/100
37/37 [==============================] - 6s 162ms/step - loss: 0.6744 - acc: 0.7166 - val_loss: 0.6402 - val_acc: 0.7358
Epoch 7/100
37/37 [==============================] - 6s 170ms/step - loss: 0.6529 - acc: 0.7273 - val_loss: 0.6307 - val_acc: 0.7499
Epoch 8/100
37/37 [==============================] - 6s 152ms/step - 

37/37 [==============================] - 6s 152ms/step - loss: 0.3824 - acc: 0.8514 - val_loss: 0.4869 - val_acc: 0.8216
Epoch 63/100
37/37 [==============================] - 6s 164ms/step - loss: 0.3852 - acc: 0.8476 - val_loss: 0.4824 - val_acc: 0.8224
Epoch 64/100
37/37 [==============================] - 6s 162ms/step - loss: 0.3789 - acc: 0.8539 - val_loss: 0.4763 - val_acc: 0.8139
Epoch 65/100
37/37 [==============================] - 6s 159ms/step - loss: 0.3824 - acc: 0.8498 - val_loss: 0.4795 - val_acc: 0.8195
Epoch 66/100
37/37 [==============================] - 6s 168ms/step - loss: 0.3800 - acc: 0.8545 - val_loss: 0.4799 - val_acc: 0.8233
Epoch 67/100
37/37 [==============================] - 6s 157ms/step - loss: 0.3826 - acc: 0.8500 - val_loss: 0.4805 - val_acc: 0.8212
Epoch 68/100
37/37 [==============================] - 6s 162ms/step - loss: 0.3740 - acc: 0.8545 - val_loss: 0.4834 - val_acc: 0.8220
Epoch 69/100
37/37 [==============================] - 6s 158ms/step - loss:

In [54]:
loss, accuracy = model_5.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model_5.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.8964
Testing Accuracy:  0.8081


### Model 6: Bidirectional RNN

In [55]:
from tensorflow.keras.layers import Bidirectional
# Bidirectional RNNs
model_6 = Sequential()
model_6.add(embedding_layer)
model_6.add(Bidirectional(LSTM(64,
                              dropout=0.2,
                              recurrent_dropout=0.5)))
model_6.add(Dense(3,activation='softmax'))
model_6.summary()


Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 100)           1576900   
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               84480     
_________________________________________________________________
dense_8 (Dense)              (None, 3)                 387       
Total params: 1,661,767
Trainable params: 84,867
Non-trainable params: 1,576,900
_________________________________________________________________


In [56]:
model_6.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
history_6 = model_6.fit(X_train, y_train,
                    validation_split = 0.2,
                    epochs=100, batch_size=256)

Epoch 1/100
37/37 [==============================] - 12s 317ms/step - loss: 0.8368 - acc: 0.6388 - val_loss: 0.7747 - val_acc: 0.6761
Epoch 2/100
37/37 [==============================] - 10s 284ms/step - loss: 0.6901 - acc: 0.7170 - val_loss: 0.6230 - val_acc: 0.7520
Epoch 3/100
37/37 [==============================] - 10s 258ms/step - loss: 0.6207 - acc: 0.7423 - val_loss: 0.6097 - val_acc: 0.7601
Epoch 4/100
37/37 [==============================] - 10s 258ms/step - loss: 0.5947 - acc: 0.7558 - val_loss: 0.5726 - val_acc: 0.7725
Epoch 5/100
37/37 [==============================] - 10s 267ms/step - loss: 0.5759 - acc: 0.7605 - val_loss: 0.6434 - val_acc: 0.7537
Epoch 6/100
37/37 [==============================] - 9s 247ms/step - loss: 0.5714 - acc: 0.7640 - val_loss: 0.5523 - val_acc: 0.7781
Epoch 7/100
37/37 [==============================] - 12s 316ms/step - loss: 0.5520 - acc: 0.7759 - val_loss: 0.5552 - val_acc: 0.7810
Epoch 8/100
37/37 [==============================] - 10s 275ms/

In [ ]:
loss, accuracy = model_6.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model_6.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))


# <font color="#004D7F">Comparación de Modelos</font>


La clave para una comparación equitativa de los algoritmos de ML es garantizar que cada algoritmo se evalúe de la misma manera en los mismos datos. Los algoritmos se comparan en un único conjunto de datos:
* Logistic Regression.
* Linear Discriminant Analysis.
* k-Nearest Neighbors.
* Classification and Regression Trees. 
* Naive Bayes.
* Support Vector Machines.

In [ ]:
models=[]
models.append(("Simple LSTM Model with regularization, increase dimensionality",  model_1.evaluate(X_train, y_train, verbose=False)))
models.append(("LSTM with regularization, reduce dimensionality",  model_2.evaluate(X_train, y_train, verbose=False)))
models.append(("LSTM Layer Stacking",  model_3.evaluate(X_train, y_train, verbose=False)))
models.append(("GRU Layer Stacking", model_4.evaluate(X_train, y_train, verbose=False)))
models.append(("Reduced GRU with More Regularization",  model_5.evaluate(X_train, y_train, verbose=False)))
models.append(("Bidirectional RNN",  model_6.evaluate(X_train, y_train, verbose=False)))

results=[]
names=[]


kfold=KFold(n_splits=10)
for name, model in models: 
    loss, accuracy = model
    results.append(accuracy)
    names.append(name)
    print(f"Accuracy: {name}: {cv_results.mean()*100.0:,.2f}%  ({cv_results.std()*100.0:,.2f}) %")